# Experiment: Check how model influence accuracy

Parameters:
- model size (number of weights / deep)
- embedding size
- training set

In [1]:
# https://huggingface.co/models?pipeline_tag=text-generation&p=1

MODELS = [
    'sshleifer/tiny-gpt2', 'lvwerra/gpt2-imdb', 'LorenzoDeMattei/GePpeTto', 'microsoft/CodeGPT-small-py', 'minimaxir/magic-the-gathering',
    'rinna/japanese-gpt2-medium', 'sberbank-ai/rugpt3small_based_on_gpt2', 'shtoshni/gpt2-chess-uci', 'uer/gpt2-chinese-poem',
    'ceostroff/harry-potter-gpt2-fanfiction', 'chrisliu298/arxiv_ai_gpt2', 'microsoft/DialoGPT-medium',
    'gpt2', 'gpt2-medium', 'gpt2-large', 'distilgpt2',
]

# TASK_NAMES = ['mnist-add-10', 'mnist-add-100', 'mnist', 'bit-xor-100']

# MODEL_SIZE = {
#     "gpt2": ...,
#     "gpt2-medium": 2_046_043,
#     'tiny-gpt2': 3_911,
# }

In [2]:
# import os
# from tqdm.notebook import tqdm

# jobs = []
# for _ in range(3):
#     for task in TASK_NAMES:
#         for model in MODELS:
#             jobs.append((_, task, model))

# with tqdm(leave=False) as progress:
#     pass
#     for _, task, model in jobs:
#         progress.set_description(f"{task}: {model}")
#         os.system(f"python3 ../run.py --tag exp-model-size --task-name {task} --model {model} ../configs/exp-model-size/gpt2.json")
    

## Collect experiments results

In [3]:
import wandb
import pandas as pd


api = wandb.Api()

# Change oreilly-class/cifar to <entity/project-name>
runs = api.runs("dl-project2/universal-computation-engine")

results = []
for run in runs:
    if 'exp-model-size' in run.tags and 'completed' in run.tags:
        state = run.name, run.config['task']+"-"+str(run.config['n']), run.config['model_name'], run.summary.get('Final Accuracy'), run.config.get('model_weights'), run.config.get('model_all_weights')
        results.append(state)

df = pd.DataFrame(results, columns=['name', 'task', 'model_name', 'final_accuracy', 'trained_weights', 'all_weights'])
df['%_weights'] = 100 * df['trained_weights'] / df['all_weights']
df

,name,task,model_name,final_accuracy,trained_weights,all_weights,%_weights
0,05-28-fpt-20210528-1241,mnist-10,sshleifer/tiny-gpt2,0.11490,84.0,102778.0,0.081730
1,05-28-fpt-20210528-1237,mnist-10,distilgpt2,0.91650,40714.0,81933322.0,0.049692
2,05-28-fpt-20210528-1234,bit-xor-10,sshleifer/tiny-gpt2,0.50059,102.0,102796.0,0.099226
3,05-28-fpt-20210528-1226,bit-xor-10,gpt2,0.94996,62228.0,124463636.0,0.049997
4,05-28-fpt-20210528-1218,bit-xor-10,uer/gpt2-chinese-poem,0.99968,62228.0,103190036.0,0.060304
...,...,...,...,...,...,...,...
97,05-27-fpt-20210527-1114,mnist-add-10,gpt2,1.98910,NaN,NaN,NaN
98,05-27-fpt-20210527-1045,mnist-add-10,gpt2,1.99940,NaN,NaN,NaN
99,05-27-fpt-20210527-1038,mnist-add-10,gpt2,2.68520,NaN,NaN,NaN
100,05-27-fpt-20210527-1025,mnist-add-10,gpt2,1.90330,NaN,NaN,NaN


In [4]:
from IPython.display import display

for task in set(df.task):
    df_task = df[df.task == task].groupby(['task', 'model_name']). \
        agg({'final_accuracy': ['size', 'mean', 'std'], 'all_weights': 'mean', 'trained_weights': ['max']}). \
        sort_values(by=('final_accuracy', 'mean'), ascending=('mnist-add' in task))
    display(df_task)

final_accuracy            \
                                                              size      mean   
task         model_name                                                        
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction              9  1.795033   
             chrisliu298/arxiv_ai_gpt2                           1  1.903600   
             distilgpt2                                          2  1.906850   
             gpt2                                                9  2.045522   
             gpt2-medium                                         5  2.116520   
             sberbank-ai/rugpt3small_based_on_gpt2               3  2.156867   
             lvwerra/gpt2-imdb                                   4  2.177775   
             microsoft/CodeGPT-small-py                          3  2.398633   
             uer/gpt2-chinese-poem                               3  2.420567   
             LorenzoDeMattei/GePpeTto                            4  2.472475   
             microsoft/DialoGPT-medium                           1  2.533500   
             gpt2-large                                          1  2.587800   
             rinna/japanese-gpt2-medium                          2  3.055950   
             minimaxir/magic-the-gathering                       8  3.286125   
             shtoshni/gpt2-chess-uci                             3  3.426467   
             sshleifer/tiny-gpt2                                 7  7.457871   

                                                               all_weights  \
                                                          std         mean   
task         model_name                                                      
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction  0.142095  125112667.0   
             chrisliu298/arxiv_ai_gpt2                    NaN  775154011.0   
             distilgpt2                              0.048013   82585435.0   
             gpt2                                    0.288062  125112667.0   
             gpt2-medium                             0.506292  355720283.0   
             sberbank-ai/rugpt3small_based_on_gpt2   0.370790  125904475.0   
             lvwerra/gpt2-imdb                       0.239575  125112667.0   
             microsoft/CodeGPT-small-py              0.185609  124916059.0   
             uer/gpt2-chinese-poem                   0.312960  103839067.0   
             LorenzoDeMattei/GePpeTto                0.519733  109555291.0   
             microsoft/DialoGPT-medium                    NaN  355720283.0   
             gpt2-large                                   NaN  775151451.0   
             rinna/japanese-gpt2-medium              0.600829  337025115.0   
             minimaxir/magic-the-gathering           0.350050    1795419.0   
             shtoshni/gpt2-chess-uci                 1.047625   86181211.0   
             sshleifer/tiny-gpt2                     0.124469     104557.0   

                                                    trained_weights  
                                                                max  
task         model_name                                              
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction       1497691.0  
             chrisliu298/arxiv_ai_gpt2                    1308251.0  
             distilgpt2                                   1479259.0  
             gpt2                                         1497691.0  
             gpt2-medium                                  2046043.0  
             sberbank-ai/rugpt3small_based_on_gpt2        2284123.0  
             lvwerra/gpt2-imdb                            1497691.0  
             microsoft/CodeGPT-small-py                   1497691.0  
             uer/gpt2-chinese-poem                        1497691.0  
             LorenzoDeMattei/GePpeTto                     1497691.0  
             microsoft/DialoGPT-medium                     997467.0  
             gpt2-large                                   1308251.0 

final_accuracy           \
                                                          size     mean   
task     model_name                                                       
mnist-10 microsoft/DialoGPT-medium                           1  0.97720   
         ceostroff/harry-potter-gpt2-fanfiction              2  0.96285   
         gpt2                                                2  0.95940   
         rinna/japanese-gpt2-medium                          1  0.95850   
         sberbank-ai/rugpt3small_based_on_gpt2               2  0.95375   
         microsoft/CodeGPT-small-py                          2  0.95065   
         lvwerra/gpt2-imdb                                   2  0.94605   
         gpt2-medium                                         1  0.94580   
         uer/gpt2-chinese-poem                               2  0.94200   
         LorenzoDeMattei/GePpeTto                            2  0.93160   
         distilgpt2                                          2  0.91540   
         shtoshni/gpt2-chess-uci                             2  0.90110   
         minimaxir/magic-the-gathering                       2  0.69795   
         sshleifer/tiny-gpt2                                 2  0.14740   

                                                           all_weights  \
                                                      std         mean   
task     model_name                                                      
mnist-10 microsoft/DialoGPT-medium                    NaN  354850826.0   
         ceostroff/harry-potter-gpt2-fanfiction  0.011667  124460554.0   
         gpt2                                    0.004384  124460554.0   
         rinna/japanese-gpt2-medium                   NaN  336155658.0   
         sberbank-ai/rugpt3small_based_on_gpt2   0.010394  125252362.0   
         microsoft/CodeGPT-small-py              0.034436  124263946.0   
         lvwerra/gpt2-imdb                       0.009405  124460554.0   
         gpt2-medium                                  NaN  354850826.0   
         uer/gpt2-chinese-poem                   0.038749  103186954.0   
         LorenzoDeMattei/GePpeTto                0.013435  108903178.0   
         distilgpt2                              0.001556   81933322.0   
         shtoshni/gpt2-chess-uci                 0.054023   85529098.0   
         minimaxir/magic-the-gathering           0.037123    1686666.0   
         sshleifer/tiny-gpt2                     0.045962     102778.0   

                                                trained_weights  
                                                            max  
task     model_name                                              
mnist-10 microsoft/DialoGPT-medium                     128010.0  
         ceostroff/harry-potter-gpt2-fanfiction         59146.0  
         gpt2                                           59146.0  
         rinna/japanese-gpt2-medium                    128010.0  
         sberbank-ai/rugpt3small_based_on_gpt2          59146.0  
         microsoft/CodeGPT-small-py                     59146.0  
         lvwerra/gpt2-imdb                              59146.0  
         gpt2-medium                                   128010.0  
         uer/gpt2-chinese-poem                          59146.0  
         LorenzoDeMattei/GePpeTto                       59146.0  
         distilgpt2                                     40714.0  
         shtoshni/gpt2-chess-uci                        59146.0  
         minimaxir/magic-the-gathering                   7818.0  
         sshleifer/tiny-gpt2                               84.0

final_accuracy            \
                                                           size      mean   
task       model_name                                                       
bit-xor-10 sberbank-ai/rugpt3small_based_on_gpt2              1  0.999910   
           microsoft/CodeGPT-small-py                         1  0.999790   
           uer/gpt2-chinese-poem                              1  0.999680   
           gpt2                                               1  0.949960   
           lvwerra/gpt2-imdb                                  2  0.699395   
           minimaxir/magic-the-gathering                      2  0.575805   
           distilgpt2                                         1  0.501630   
           sshleifer/tiny-gpt2                                2  0.500780   
           shtoshni/gpt2-chess-uci                            1  0.499500   

                                                            all_weights  \
                                                       std         mean   
task       model_name                                                     
bit-xor-10 sberbank-ai/rugpt3small_based_on_gpt2       NaN  125255444.0   
           microsoft/CodeGPT-small-py                  NaN  124267028.0   
           uer/gpt2-chinese-poem                       NaN  103190036.0   
           gpt2                                        NaN  124463636.0   
           lvwerra/gpt2-imdb                      0.283571  124463636.0   
           minimaxir/magic-the-gathering          0.104334    1687188.0   
           distilgpt2                                  NaN   81936404.0   
           sshleifer/tiny-gpt2                    0.000269     102796.0   
           shtoshni/gpt2-chess-uci                     NaN   85532180.0   

                                                 trained_weights  
                                                             max  
task       model_name                                             
bit-xor-10 sberbank-ai/rugpt3small_based_on_gpt2         62228.0  
           microsoft/CodeGPT-small-py                    62228.0  
           uer/gpt2-chinese-poem                         62228.0  
           gpt2                                          62228.0  
           lvwerra/gpt2-imdb                             62228.0  
           minimaxir/magic-the-gathering                  8340.0  
           distilgpt2                                    43796.0  
           sshleifer/tiny-gpt2                             102.0  
           shtoshni/gpt2-chess-uci                       62228.0

In [5]:
# for r in results:
#     if r[1] == 'mnist-add-10' and r[2] == 'ceostroff/harry-potter-gpt2-fanfiction':
#         print(r)


In [6]:
# run.config
# run.summary.keys()
# run.name